In [ ]:
#Recompensa 1, penalización -0.1
import numpy as np
import random

# Definir el tamaño del rompecabezas
num_filas = 4
num_columnas = 5

# Definir las acciones posibles (mover el espacio vacío: arriba, abajo, izquierda, derecha)
acciones = ['arriba', 'abajo', 'izquierda', 'derecha']

# Función para mover el espacio vacío en la dirección dada
def mover(puzzle, direccion):
    new_puzzle = puzzle.copy()
    espacio_fila, espacio_columna = np.argwhere(new_puzzle == 0)[0]

    if direccion == 'arriba' and espacio_fila > 0:
        new_puzzle[espacio_fila, espacio_columna], new_puzzle[espacio_fila - 1, espacio_columna] = new_puzzle[espacio_fila - 1, espacio_columna], new_puzzle[espacio_fila, espacio_columna]
    elif direccion == 'abajo' and espacio_fila < num_filas - 1:
        new_puzzle[espacio_fila, espacio_columna], new_puzzle[espacio_fila + 1, espacio_columna] = new_puzzle[espacio_fila + 1, espacio_columna], new_puzzle[espacio_fila, espacio_columna]
    elif direccion == 'izquierda' and espacio_columna > 0:
        new_puzzle[espacio_fila, espacio_columna], new_puzzle[espacio_fila, espacio_columna - 1] = new_puzzle[espacio_fila, espacio_columna - 1], new_puzzle[espacio_fila, espacio_columna]
    elif direccion == 'derecha' and espacio_columna < num_columnas - 1:
        new_puzzle[espacio_fila, espacio_columna], new_puzzle[espacio_fila, espacio_columna + 1] = new_puzzle[espacio_fila, espacio_columna + 1], new_puzzle[espacio_fila, espacio_columna]

    return new_puzzle

# Función para determinar si el estado es el objetivo
def es_estado_objetivo(puzzle):
    objetivo = np.arange(1, num_filas * num_columnas).tolist() + [0]
    return np.array_equal(puzzle.flatten(), objetivo)

# Inicializar Q-table
Q = {}

# Hiperparámetros
alpha = 0.5 #tasa de aprendizaje, rapidez con la que se actualizan los valor de q en cada iteración
gamma = 0.9 #factor de descuento, el agente se enfoca mas en obtener recompensas mas grandes a largo plazo
epsilon = 1 #probabilidad de exploración, reduce gradualmente,

# Función para elegir una acción usando epsilon-greedy, a veces acción aleatoria para explorar rutas, a veeces la accion con mayor valor
def elegir_accion(puzzle):
    estado = str(puzzle)
    if np.random.uniform(0, 1) < epsilon or estado not in Q:
        return random.choice(acciones)
    else:
        return max(Q[estado], key=Q[estado].get)

# Bucle de entrenamiento, hiperparametro
num_episodios = 1

for episodio in range(num_episodios):
    # Inicializar el rompecabezas de manera aleatoria
    puzzle = np.arange(num_filas * num_columnas).reshape(num_filas, num_columnas)
    np.random.shuffle(puzzle.flat)

    while not es_estado_objetivo(puzzle):
        estado = str(puzzle)
        accion = elegir_accion(puzzle)
        nuevo_puzzle = mover(puzzle, accion)
        nuevo_estado = str(nuevo_puzzle)
        #se actualiza la tabla Q después de cada movimiento del rompecabezas
        if nuevo_estado not in Q:
            Q[nuevo_estado] = {a: 5.0 for a in acciones}

        recompensa = 1.0 if es_estado_objetivo(nuevo_puzzle) else -0.1

        if estado not in Q:
            Q[estado] = {a: 5.0 for a in acciones}

        Q[estado][accion] += alpha * (recompensa + gamma * max(Q[nuevo_estado].values()) - Q[estado][accion])

        puzzle = nuevo_puzzle

# Función para imprimir la Q-table (truncada para los primeros estados)
def imprimir_q_table(Q, num_estados=5):
    for i, estado in enumerate(Q.keys()):
        if i >= num_estados:
            break
        print(f"Estado: {estado}")
        for accion in Q[estado]:
            print(f"  Acción {accion}: {Q[estado][accion]:.2f}")
        print()

# Imprimir la Q-table al finalizar el entrenamiento
imprimir_q_table(Q)


<ipython-input-10-93a681baa41a>:55: UserWarning: you are shuffling a 'flatiter' object which is not a subclass of 'Sequence'; `shuffle` is not guaranteed to behave correctly. E.g., non-numpy array/tensor objects with view semantics may contain duplicates after shuffling.
  np.random.shuffle(puzzle.flat)
